In [24]:

import os
import random
import time
from random import shuffle


import matplotlib.pyplot as plt
import numpy as np
import pandas as pd
import tensorflow as tf
#import numpy as np
#import tensorflow as tf

from keras.layers import Activation
from keras.utils.generic_utils import get_custom_objects
from sklearn import preprocessing
from tensorflow import keras
from tqdm import tqdm

pd.set_option ('display.max_columns',None)


def setfig (colume = 1):
	'''
    在绘图前对字体类型、字体大小、分辨率、线宽、输出格式进行设置.
    para colume = 1.半栏图片 7*7cm
                  2.双栏长图 14*7cm
    x轴刻度默认为整数
    手动保存时，默认输出格式为 pdf

    案例 Sample.1:
    setfig(colume=1) # 参数预设置
    plt.plot(x, color='red', linestyle='dashed', label='lengend_label') #此处label指的是Lengend或图例名称
    plt.xlabel('x_axis')
    plt.ylabel('y_axis')
    plt.title('figure_title')
    plt.legend(loc='upper left')
    plt.tight_layout()  #此项必需，以保证绘图正常
    '''
	cm_to_inc = 1 / 2.54  # 厘米和英寸的转换 1inc = 2.54cm
	if colume == 1:
		plt.rcParams['figure.figsize'] = (7 * cm_to_inc,6 * cm_to_inc)  #单位 inc
	elif colume == 2:
		plt.rcParams['figure.figsize'] = (14 * cm_to_inc,6 * cm_to_inc)
	else:
		pass

	# 对尺寸和 dpi参数进行调整
	plt.rcParams['figure.dpi'] = 300

	# 字体调整
	plt.rcParams['font.sans-serif'] = ['SimHei']  # 如果要显示中文字体,则在此处设为：SimHei
	plt.rcParams['font.weight'] = 'light'
	plt.rcParams['axes.unicode_minus'] = False  # 坐标轴负号显示
	plt.rcParams['axes.titlesize'] = 8  # 标题字体大小
	plt.rcParams['axes.labelsize'] = 7  # 坐标轴标签字体大小
	plt.rcParams['xtick.labelsize'] = 7  # x轴刻度字体大小
	plt.rcParams['ytick.labelsize'] = 7  # y轴刻度字体大小
	plt.rcParams['legend.fontsize'] = 6

	# 线条调整
	plt.rcParams['axes.linewidth'] = 1

	# 刻度在内，设置刻度字体大小
	plt.rcParams['xtick.direction'] = 'in'
	plt.rcParams['ytick.direction'] = 'in'

	# 设置输出格式为PDF
	plt.rcParams['savefig.format'] = 'svg'

	# 设置坐标轴范围：如需要可在函数外进行设置
	from matplotlib.ticker import MaxNLocator
	plt.gca ().xaxis.set_major_locator (MaxNLocator (integer=True))  # x轴刻度设置为整数


plt.rcParams['font.sans-serif'] = ['SimHei']  # 显⽰中文
plt.rcParams['axes.unicode_minus'] = False  # 显⽰负号


def get_str_time ():
	s = '_'
	for i in time.localtime (time.time ())[0:6]:
		s += str (i) + '_'
	return s


def make_file (file_dir):
	if not os.path.exists (file_dir):
		os.makedirs (file_dir)

def get_csv(file_path):
	"""
	得到所有csv数据路径
	"""
	root, dir, file = None, None, None
	for root, dir, file in os.walk(file_path):
		root, dir, file = root, dir, file
	file_paths = []
	for i in file:
		a = os.path.join(root, i)
		file_paths.append(a)
	return file_paths


def get_columns_single (file_paths):
	"""
	得到所有测点名的交集
	"""
	result = []
	for path in file_paths:
		data_2 = pd.read_csv (path)#,encoding='gbk')
		result.append (set (data_2.columns.tolist ()))

	columns_single = result[0]
	for i in result[1:]:
		columns_single = columns_single & i
	return columns_single


def get_single_table (path,file_paths):
	"""
	得到单张数据表的数据
	"""
	data = pd.read_csv (path)#,encoding='gbk')
	temp_dataframe = pd.DataFrame ()
	columns_single = get_columns_single (file_paths)
	for column in columns_single:
		temp_dataframe[column] = data[column]
	return temp_dataframe


def get_all_corr_map(file_paths):
	"""
	绘制所有变量的相关性热力图
	"""
	result=[]
	for path in file_paths:
		pdframe = get_single_table(path,file_paths)
		result.append(pdframe)
	total=pd.concat(result)
	plt.rcParams['axes.unicode_minus']=False

	plt.figure(figsize=(30, 30))
	plt.yticks(fontproperties='Times New Roman', size=25,weight='bold')#设置大小及加粗
	plt.xticks(fontproperties='Times New Roman', size=25,weight='bold')
	plt.imshow(np.corrcoef(total.T), 'cool')#,#cmap='cool',alpha=1)
	cbar=plt.colorbar()
	#cbar.set_label("热力值",fontsize=22)
	#cbar.formatter.set_scientific(True)
	#cbar.formatter.set_powerlimits((0,0))
	cbar.ax.tick_params(labelsize=40) #改变bar标签字体⼤⼩
	#cbar.ax.yaxis.get_offset_text().set_fontsize(18) #改变bar顶部字体⼤⼩
	cbar.update_ticks()

	plt.title('相关性热力图',fontsize=40)
	plt.xlabel("参数",fontsize=40)
	plt.ylabel("参数",fontsize=40)
	plt.savefig(r"D:\file_comb\研究生毕业论文\中期检查\ppt\图片\相关性.svg",dpi=300, bbox_inches='tight')
	plt.show()


def get_special_data(data):
	"""
	返回传入数据的各个参数
	"""
	result = pd.DataFrame()
	result["一回路压力平均值"] = (data['rcp037mp'] + data['rcp039mp']) / 2
	result["一环路冷热段平均值"] = (data['3rcp028mt'] + data['3rcp029mt']) / 2
	result["堆芯出口温度平均值"] = (data['3ric100km'] + data['3ric200km']) / 2
	result["主蒸汽压力平均值"] = (data['vvp011mp'] + data['vvp014mp'] + data['vvp008mp'] + data['vvp009mp']) / 4
	#负相关
	result['辅助给水流量平均值'] = (data['asg012km'] + data['asg013km']) / 2
	result['重要厂用水系统压力'] = data['sec004mp']
	result['DeltaTsat平均值'] = (data['3ric103km'] + data['3ric203km']) / 2
	result['安全壳压力平均值'] = (data['ety103mp'] + data['ety104mp']) / 2
	result['SG1水位平均值'] = (data['are052mn'] + data['are010mn'] + data['are055mn'] + data['are058mn']) / 4
	result['稳压器泄压箱压力'] = data['3rcp024mp_a_pnt']
	result['压缩空气系统压力'] = data['sap001mp']
	#动作
	result["汽机旁路系统蒸汽阀平均开度"] = (data['3gct131vv_lc_opnt'] + data['3gct132vv_lc_opnt'] + data['3gct133vv_lc_opnt']) / 3
	#result["action_average"] = result["汽机旁路系统蒸汽阀平均开度"]
	return result


def get_all_data(file_paths,final_feature=False):
	"""
	返回列表，包含所有数据表的特征和奖励
	"""
	result = []
	for path in file_paths:
		pdframe = get_single_table(path,file_paths)
		pdframe.convert_dtypes()
		if final_feature:
			new_pdframe = pdframe[final_feature[0:-2]]
			new_pdframe["first_loop_average_tmp"] = (new_pdframe['3rcp028mt'] + new_pdframe['3rcp029mt']) / 2
			first_loop_average_tmp = new_pdframe['first_loop_average_tmp'].tolist()
			temp_change_speed = []
			last_tmp = first_loop_average_tmp[0]
			for i in first_loop_average_tmp:
				speed = (i - last_tmp) * 3600
				temp_change_speed.append(speed)
				last_tmp = i
			new_pdframe["temp_change_speed"] = temp_change_speed
			temp_change_speed = new_pdframe["temp_change_speed"].tolist()
			rewards = []
			for i in temp_change_speed:
				reward = 100 - abs(i - (-56))
				rewards.append(reward)
			new_pdframe["action_average"] = (new_pdframe['3gct131vv_lc_opnt'] + new_pdframe['3gct132vv_lc_opnt'] +
											 new_pdframe['3gct133vv_lc_opnt']) / 3
			new_pdframe["reward"] = rewards
			result.append(new_pdframe)
		else:
			new_pdframe = get_special_data(pdframe)
			first_loop_average_tmp = new_pdframe['一环路冷热段平均值'].tolist()
			temp_change_speed = []
			last_tmp = first_loop_average_tmp[0]
			for i in first_loop_average_tmp:
				speed = (i - last_tmp) * 3600
				temp_change_speed.append(speed)
				last_tmp = i
			rewards = []
			for i in temp_change_speed:
				reward = 100 - abs(i - (-56))
				rewards.append(reward)
			new_pdframe["动作"] = new_pdframe["汽机旁路系统蒸汽阀平均开度"]
			new_pdframe["奖励"] = rewards

			result.append(new_pdframe)

	return result


def offset_act_rew(data):
	"""
	偏移数据表的动作和奖励
	"""
	results = []
	for i in range(len(data)):
		action = data[i]["动作"].to_list()
		action.pop(0)
		action.append(0)
		data[i]["动作"].iloc[:] = action
		reward = data[i]["奖励"].to_list()
		reward.pop(0)
		reward.append(0)
		data[i]["奖励"].iloc[:] = reward
		result = data[i].drop(index=(len(data[i]) - 1))
		results.append(result)
	return results


def get_normalized_dataframe(total_data_frame1):
	# 返回传入数据表的标准化形式
    std_scaler=preprocessing.StandardScaler().fit(total_data_frame1)
    total_data_frame2=std_scaler.transform(total_data_frame1)
    total_data_frame2=pd.DataFrame(total_data_frame2,columns=total_data_frame1.columns)
    return total_data_frame2


def get_start_end(pdframe):
    """
    返回传入数据表列表的开头和结尾位置
    """
    len_result=[]
    for i in pdframe:
        len_result.append(len(i))

    start,end=0,0
    result=[]
    for num,i in enumerate(len_result):
        start=end
        end=start+i
        result.append((start,end))
    return result


def get_sars_total_normalized_data(offset_data,start_end):
    """
    得到标准化后的sars元组数据
    pd_results：标准化后的各个数据表
    total_len：标准化后总数据长度
    features_len：特征个数
    """
    total_data_frame=pd.concat(offset_data,ignore_index=True)
    std_data=get_normalized_dataframe(total_data_frame)
    total_len=len(std_data)
    total_feature_len=len(total_data_frame.columns)
    features_len=total_feature_len-2
    pd_results=[]
    for start,end in start_end:
        result=[]
        for row in range(start,end):
            sars={}
            rows_data=std_data.iloc[row,:].to_list()
            s=rows_data[0:features_len]
            a=rows_data[features_len]
            r=rows_data[total_feature_len-1]
            if row<end-1:
                s_=std_data.iloc[row+1,:].to_list()[0:features_len]
                sars["s"],sars["a"],sars["r"],sars["s_"]=s,a,r,s_
                result.append(sars)
        pd_results.append(result)
    return pd_results,total_len,features_len


def get_action_minmax(offset_data):
	"""
	返回动作的区间
	"""
	total_data_frame=pd.concat(offset_data,ignore_index=True)
	std_data=get_normalized_dataframe(total_data_frame)
	action_max=max(std_data["动作"].values)
	action_min=min(std_data["动作"].values)
	return action_max,action_min

In [25]:

file_paths = get_csv(r"E:\file_comb\核二院\数据\wuhan\数据采集0421\SGTR")
#final_feature=pd.read_csv("final_feature.csv")["0"].tolist()
alldata_list = get_all_data(file_paths)
alldata_list

M:\work\softfiles\Anaconda3\envs\tensorflow_gpu_version\lib\site-packages\ipykernel_launcher.py:128: PerformanceWarning: DataFrame is highly fragmented.  This is usually the result of calling `frame.insert` many times, which has poor performance.  Consider joining all columns at once using pd.concat(axis=1) instead.  To get a de-fragmented frame, use `newframe = frame.copy()`


[      一回路压力平均值  一环路冷热段平均值  堆芯出口温度平均值  主蒸汽压力平均值  辅助给水流量平均值   重要厂用水系统压力  \
 0     15.38940   309.8145    333.440  6.670185      0.000    0.012031   
 1     15.38710   309.8350    333.435  6.670212      0.000    0.012031   
 2     15.37960   309.8325    333.431  6.670265      0.000    0.012031   
 3     15.37100   309.8280    333.427  6.670345      0.000    0.012031   
 4     15.36540   309.8290    333.423  6.670438      0.000    0.012031   
 ...        ...        ...        ...       ...        ...         ...   
 4497   4.40247   227.8230    228.022  2.291215     38.947  194.972000   
 4498   4.40247   227.8230    228.022  2.291215     38.947  194.972000   
 4499   4.40247   227.8230    228.022  2.291215     38.947  194.972000   
 4500   4.40247   227.8230    228.022  2.291215     38.947  194.972000   
 4501   4.40247   227.8230    228.022  2.291215     38.947  194.972000   
 
       DeltaTsat平均值  安全壳压力平均值  SG1水位平均值  稳压器泄压箱压力  压缩空气系统压力  汽机旁路系统蒸汽阀平均开度  \
 0          11.1279   100.982  0

In [26]:
alldata_list[0]

,一回路压力平均值,一环路冷热段平均值,堆芯出口温度平均值,主蒸汽压力平均值,辅助给水流量平均值,重要厂用水系统压力,DeltaTsat平均值,安全壳压力平均值,SG1水位平均值,稳压器泄压箱压力,压缩空气系统压力,汽机旁路系统蒸汽阀平均开度,动作,奖励
0,15.38940,309.8145,333.440,6.670185,0.000,0.012031,11.1279,100.982,0.000157,0.015751,0.815506,0.000000,0.000000,44.0
1,15.38710,309.8350,333.435,6.670212,0.000,0.012031,11.0896,100.982,0.001192,0.015750,0.815518,0.000000,0.000000,-29.8
2,15.37960,309.8325,333.431,6.670265,0.000,0.012031,11.0466,100.982,0.002432,0.015747,0.815531,0.000000,0.000000,53.0
3,15.37100,309.8280,333.427,6.670345,0.000,0.012031,11.0160,100.982,0.003747,0.015747,0.815543,0.000000,0.000000,60.2
4,15.36540,309.8290,333.423,6.670438,0.000,0.012031,10.9900,100.982,0.005079,0.015750,0.815555,0.000000,0.000000,40.4
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
4497,4.40247,227.8230,228.022,2.291215,38.947,194.972000,29.2856,101.121,1.800000,0.040001,0.871307,43.127167,43.127167,44.0
4498,4.40247,227.8230,228.022,2.291215,38.947,194.972000,29.2856,101.121,1.800000,0.040001,0.871307,43.127167,43.127167,44.0
4499,4.40247,227.8230,228.022,2.291215,38.947,194.972000,29.2856,101.121,1.800000,0.040001,0.871307,43.127167,43.127167,44.0
4500,4.40247,227.8230,228.022,2.291215,38.947,194.972000,29.2856,101.121,1.800000,0.040001,0.871307,43.127167,43.127167,44.0


In [27]:
offset_data = offset_act_rew(alldata_list)
offset_data[0]

,一回路压力平均值,一环路冷热段平均值,堆芯出口温度平均值,主蒸汽压力平均值,辅助给水流量平均值,重要厂用水系统压力,DeltaTsat平均值,安全壳压力平均值,SG1水位平均值,稳压器泄压箱压力,压缩空气系统压力,汽机旁路系统蒸汽阀平均开度,动作,奖励
0,15.38940,309.8145,333.440,6.670185,0.000,0.012031,11.1279,100.982,0.000157,0.015751,0.815506,0.000000,0.000000,-29.8
1,15.38710,309.8350,333.435,6.670212,0.000,0.012031,11.0896,100.982,0.001192,0.015750,0.815518,0.000000,0.000000,53.0
2,15.37960,309.8325,333.431,6.670265,0.000,0.012031,11.0466,100.982,0.002432,0.015747,0.815531,0.000000,0.000000,60.2
3,15.37100,309.8280,333.427,6.670345,0.000,0.012031,11.0160,100.982,0.003747,0.015747,0.815543,0.000000,0.000000,40.4
4,15.36540,309.8290,333.423,6.670438,0.000,0.012031,10.9900,100.982,0.005079,0.015750,0.815555,0.000000,0.000000,35.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
4496,4.40247,227.8230,228.022,2.291215,38.947,194.972000,29.2856,101.121,1.800000,0.040001,0.871307,43.127167,43.127167,44.0
4497,4.40247,227.8230,228.022,2.291215,38.947,194.972000,29.2856,101.121,1.800000,0.040001,0.871307,43.127167,43.127167,44.0
4498,4.40247,227.8230,228.022,2.291215,38.947,194.972000,29.2856,101.121,1.800000,0.040001,0.871307,43.127167,43.127167,44.0
4499,4.40247,227.8230,228.022,2.291215,38.947,194.972000,29.2856,101.121,1.800000,0.040001,0.871307,43.127167,43.127167,44.0


In [28]:
start_end=get_start_end(offset_data)
start_end

[(0, 4501),
 (4501, 8517),
 (8517, 12117),
 (12117, 15720),
 (15720, 18647),
 (18647, 22154),
 (22154, 27011),
 (27011, 30489)]

In [29]:
total_normalized_data,total_len,features_len=get_sars_total_normalized_data(offset_data,start_end)
total_normalized_data[0]

[{'s': [2.841947896208951,
   2.0002769030129475,
   2.4696092399451346,
   1.3049012315367294,
   -1.9852505987601419,
   -3.311317182797717,
   -1.7205633065943144,
   -3.2061515300574817,
   -1.1928446295507695,
   -1.4330557442200464,
   -1.7340359430843792,
   -1.1461202821541692],
  'a': -1.146660695257754,
  'r': -0.3744526150756887,
  's_': [2.841104130727466,
   2.0011204966484906,
   2.4694330095150905,
   1.3049180774724745,
   -1.9852505987601419,
   -3.311317182797717,
   -1.7249311574958535,
   -3.2061515300574817,
   -1.1917269855589676,
   -1.4332779713449233,
   -1.7332115297297734,
   -1.1461202821541692]},
 {'s': [2.841104130727466,
   2.0011204966484906,
   2.4694330095150905,
   1.3049180774724745,
   -1.9852505987601419,
   -3.311317182797717,
   -1.7249311574958535,
   -3.2061515300574817,
   -1.1917269855589676,
   -1.4332779713449233,
   -1.7332115297297734,
   -1.1461202821541692],
  'a': -1.146660695257754,
  'r': -0.0005360604413063455,
  's_': [2.8383527215

In [30]:
get_action_minmax(offset_data)

(4.140236416412629, -1.146660695257754)

In [31]:

total_normalized_sars_data=[]
for i in total_normalized_data:
	total_normalized_sars_data.extend(i)


In [32]:
total_normalized_sars_data.__len__()

30481

In [33]:
dataa=np.array(total_normalized_sars_data,dtype='object')
np.save ("全部sars数据.npy",dataa)

In [59]:
aaa=np.array([1])
aaa.shape

(1,)